## [5주차] 기본과제 - 외부 블로그의 정보와 함께 챗봇 구현하기

- [internet source](https://spartacodingclub.kr/blog/all-in-challenge_winner)
- Question: “ALL-in 코딩 공모전 수상작들을 요약해줘.”

In [2]:
%pip install dotenv langchain-community langchain-chroma langchain-openai bs4

  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.


API KEY를 불러온다.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

모델은 gpt-4o-mini를 사용한다.

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

knowledge source로 사용할 웹 사이트 url를 지정하고, bs4를 사용하여 웹 사이트의 내용을 불러온다.

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

source_url="https://spartacodingclub.kr/blog/all-in-challenge_winner"

loader = WebBaseLoader(
    web_paths=(source_url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("css-j3idia", "editedContent")
        )
    ),
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


데이터를 잘 가져왔는지, 데이터의 크기는 어떤지 확인해본다.

In [6]:
# 문서 크기 확인
for i, doc in enumerate(docs):
    print(f"문서 {i+1} 길이: {len(doc.page_content)} 글자")
    # 첫 100자 미리보기
    print(f"미리보기: {doc.page_content[:100]}...\n")

문서 1 길이: 5430 글자
미리보기: 'AII-in 코딩 공모전’ 수상작을 소개합니다코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩...



문서의 크기를 고려하여 chuck size와 chunk overlap을 설정한다.
chuck overlap은 chunk size의 20%로 설정한다.

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 각 청크의 최대 크기 (문자 수)
    chunk_overlap=300 # 청크 간 중복되는 문자 수
)

# 문서를 청크로 분할
splits = text_splitter.split_documents(docs)

# Chroma 벡터 데이터베이스 생성
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

rag-prompt를 사용하여 prompt를 작성한다.

In [8]:
from langchain import hub

retriever = vectorstore.as_retriever()

user_msg="ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

/opt/anaconda3/envs/llm/lib/python3.12/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 'AII-in 코딩 공모전’ 수상작을 소개합니다코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 

응답은 다음과 같다.
주요 수상작과, 공모전의 특징을 짧지만 잘 요약해서 답변해준다.

In [9]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전 수상작 중, 대상은 언어 학습을 효율적으로 돕는 웹 서비스 'Lexi Note'가 선정되었습니다. 우수상에는 벌레 퇴치 매칭 서비스 '우리집 히어로즈'와 수업 실시간 소통 플랫폼 '에코 클래스룸'이 있으며, 입선작으로는 연합 동아리 정보 플랫폼 'Crewing'과 학교생활 관리 앱 '학교생활 매니저'가 포함됩니다. 이 공모전은 대학생들이 캠퍼스에서의 문제를 창의적으로 해결하기 위해 만든 다양한 프로젝트를 소개합니다.
